# Word2Vec using Skip Gram Model

In [1]:
import tensorflow as tf
import os
import sys
import zipfile
from collections import Counter
import numpy as np
from six.moves import urllib
import random

### 下载数据

In [2]:
DOWNLOAD_URL = "http://mattmahoney.net/dc/text8.zip"      # 下载文件的URL
DATA_FOLDER = "./data/"                                   # 存放数据文件的文件夹路径
FILE_NAME = "text8.zip"                                   # 数据文件的名称
EXPECTED_BYTES = 31344016                                 # 文件的 bytes 大小

In [3]:
# 定义一个创建本地文件夹的函数
# 
# 参数
# path              : 创建路径
#
def make_dir(path):
    try:
        os.mkdir(path)
    except OSError:
        pass
    
# 获得当前文件的 bytes 大小
# 
# 参数
# file_path         : 文件的路径
#
# return            : 当前文件的大小
# 
def get_bytes(file_path):
    
    # 获得文件的描述性数据
    file_stats = os.stat(file_path)
    
    # 返回文件的大小
    return file_stats.st_size
    
# 检查数据的大小是否正确，用来检查是否下载了 “ 完整 ” 的数据集
# 如果文件大小不符合所期待的大小，则抛出异常
# 
# 参数
# file_path         : 文件路径
# expected_bytes    : 所期待的文件的大小
#
def check_bytes(file_path, expected_bytes):
    
    # 如果不符合期待的文件大小，则抛出异常
    if get_bytes(file_path) != expected_bytes:
        raise Exception("File has not been download successfully, please try download again.")

In [4]:
# 定义一个下载数据的函数，并检查下载的数据是否完整的被下载了
#
# 参数
# source_url        : 文件 URL 下载路径
# download_folder   : 下载到本地 文件夹 的名字
# file_name         : 文件名
# expected_bytes    : 文件大小
#
# return            : 文件的路径
#
def download(download_url   = DOWNLOAD_URL, 
             data_folder    = DATA_FOLDER, 
             file_name      = FILE_NAME, 
             expected_bytes = EXPECTED_BYTES):
    
    # 如果下载数据的路径不存在的时候，则创建一个
    if not os.path.exists(data_folder):
        make_dir(data_folder)
        
    # 下载的数据的路径为文件夹的路径 + 文件名
    file_path = data_folder + file_name
    
    # 如果文件已经存在
    if os.path.exists(file_path):
        # 检查文件是否完整
        if get_bytes(file_path) == expected_bytes:
            # 如果完整，则返回该文件的路径，不做接下来的处理了
            print("Dataset already downloaded.")
            return file_path
        else:
            # 如果文件不完整，则删除文件，接下来重新下载一次
            os.remove(file_path)
    
    # 从网页上下载数据，下载文件可能会需要一段时间，请耐心等待
    print("Start downloading the data, the process may take several minutes, please be patient...")
    file_name, _ = urllib.request.urlretrieve( url = download_url, filename = file_path )
    
    # 检查下载的数据是否完整
    check_bytes( file_path, expected_bytes )
    
    # 返回文件的路径
    return file_path

In [5]:
# 下载数据
file_path = download()

# 检查数据的完整性
check_bytes( file_path, EXPECTED_BYTES )

# 下载成功
print("File downloaded at path", file_path)

Dataset already downloaded.
File downloaded at path ./data/text8.zip


### 读取数据

In [6]:
# 从zip中读取所有的单词
# 参数
# file_path         : ZIP 文件的路径
#
# return            : 该文件所包含的所有单词
#
def read_data(file_path):
    with zipfile.ZipFile( file = file_path ) as f:
        # namelist : 返回在压缩目录下的所有文件
        # read     : 读出文件的 bytes
        words = tf.compat.as_str( f.read( f.namelist()[0] ) ).split()
    return words

In [29]:
# 读取单词
words = read_data(file_path)

# 打印读取的单词的长度
print( "The whole content contains {} words.".format( len(words) ) )

# 打印最开始的5个单词
print( "The first 5 words are : {}.".format( words[:5]) )

The whole content contains 17005207 words.
The first 5 words are : ['anarchism', 'originated', 'as', 'a', 'term'].


### 构建数据集：将所有单词转换为index

In [16]:
# 定义词库的大小为 50000
VOCAB_SIZE = 50000

In [17]:
# 构建一个 word -> index 的 dictionary，
# 以及一个 index -> word 的 reverse_dictionary
# 参数
# words             : 单词输入，用来创建dictionary
# vocab_size        : 词库的大小
# 
# return 
#   word_index          :  输入的单词序列 转换成的 index 的序列
#                            [ 36, 1, 0, 998, ... , 12 ]
#   count               :  一个长度为 词库大小 的数组，每个数组的元素为 [word, count]
#                            [ ['UNK', 3456789], ['the', 12345], ['a',12344], ... , ]
#   dictionary          :  word -> index 的 dictionary
#   reverse_dictionary  :  index -> word 的 reverse_dictionary
#   
def build_dataset(words, vocab_size):
    
    dictionary = {}              # 初始化空词库
    count = [['UNK',-1]]         # 初始化 Unknown 的单词计数为-1
    
    # 找出最出现的单词，加入到count数组中
    # 单词的个数为 词库的大小 减一，因为有一个位置已经被 unknown 占据了
    count.extend(Counter(words).most_common(vocab_size-1))
    
    index = 0                    # 用来记录每个单词在词库中的 index
    
    # 创建一个目录来存放前 1000 个单词的
    make_dir("processed")
    with open("processed/vocab_1000.tsv", "w") as f:
        for word, _ in count:
            # 遍历 count 来生成 word -> index 的 dictionary
            dictionary[word] = index
            # 将前 1000 个单词写入文件中
            if index < 1000:
                f.write(word + '\n')
            index += 1
    
    # 将所有的从 zip 文件中读取的单词，转换成相对应的 index
    # 如果单词存在于词库中，则返回它的index值
    # 否则，返回 0 -- UNK
    # 最终这个 len(word_index) == len(words)
    word_index = [dictionary[word] if word in dictionary else 0 for word in words]
    
    # word_index 中 0 出现的次数，即是 UNK 单词出现的次数
    count[0][1] = word_index.count(0)
    
    # zip  : zip 函数会对传入的 iterables 进行遍历和组合，
    #         最终返回 合并到一起 的 tuple 的数组
    #         # zip('ABCD', 'xy') --> Ax By
    #
    # dict : dict 函数可以从 tuple 的数组中生成一个新的dictionary
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    
    # 返回 单词的对应的 index 序列，每个单词对应出现的次数表，单词 - index 表， index - 单词 表
    return word_index, count, dictionary, reverse_dictionary

In [30]:
# 计算 word_index, count, dictionary, reverse_dictionary
word_index, count, dictionary, reverse_dictionary = build_dataset(words, VOCAB_SIZE)

# 可以节省内存
del words

# 查看一下嘴常见的 5 个单词
print("5 most common words are : {}.\n".format( count[:5] ) )

# 看一下 word_index 中的前十个 index 的值 和 对应的单词
print("10 first index and word are : {}.".format( ', '.join(
    str(index) + '/' +(reverse_dictionary[index]) for index in word_index[:10] ) ) )

5 most common words are : [['UNK', 418391], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764)].

10 first word and index are : 5234/anarchism, 3081/originated, 12/as, 6/a, 195/term, 2/of, 3134/abuse, 46/first, 59/used, 156/against.


### 生成训练的 batch